In [11]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Đọc dữ liệu
file_path = "D:/LeHuuKieu.csv"
df = pd.read_csv(file_path)

# Chuyển đổi cột Date thành datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Tạo khoảng ngày đầy đủ
date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max())

# Đưa dữ liệu về index theo ngày
df.set_index('Date', inplace=True)
df = df.reindex(date_range).reset_index().rename(columns={'index': 'Date'})

# Hàm điền dữ liệu thiếu bằng KNN
def knn_impute(df, column_name):
    df['date_ordinal'] = df['Date'].map(pd.Timestamp.toordinal)
    knn_imputer = KNNImputer(n_neighbors=1)
    imputed_data = knn_imputer.fit_transform(df[['date_ordinal', column_name]])
    df[column_name] = imputed_data[:, 1]
    df.drop(columns=['date_ordinal'], inplace=True)
    return df

# Áp dụng KNN cho từng cột
columns_to_impute = ['PM25', 'PM10', 'CO', 'SO2', 'NO2', 'O3']
for col in columns_to_impute:
    df = knn_impute(df, col)

# Chuẩn hóa dữ liệu bằng MinMaxScaler
scaler = MinMaxScaler()
for col in columns_to_impute:
    df[f'{col}_scaled'] = scaler.fit_transform(df[[col]])

# Xuất dữ liệu sau xử lý
df.to_csv("D:/LeHuuKieu_processed.csv", index=False)

In [16]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Hàm tải và xử lý tệp dữ liệu
def load_and_process_file(file_path, column_names):
    file_path = file_path.strip('\u202a')
    data = pd.read_csv(file_path, encoding='utf-8-sig', delimiter=',', decimal='.', header=None, skiprows=1, names=column_names)
    data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
    data = data.sort_values(by='Date')
    return data

# Hàm điền các ngày bị thiếu
def fill_missing_dates(df, column_name):
    start_date = pd.to_datetime('2024-01-01')
    end_date = pd.to_datetime('2025-03-03')
    full_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    missing_dates = full_dates.difference(df['Date'])
    missing_df = pd.DataFrame({'Date': missing_dates, column_name: np.nan})
    df = pd.concat([df, missing_df], ignore_index=True).sort_values('Date').reset_index(drop=True)
    return df

# Đọc dữ liệu
file_path = "D:/LeHuuKieu.csv"
column_names = ["Ten_Tram", "Nguon", "Link", "KinhDo_WGS84", "ViDo_WGS84", "KinhDo_VN2000", "ViDo_VN2000", "Date", "PM25", "PM10", "CO", "SO2", "NO2", "O3"]
df = load_and_process_file(file_path, column_names)

# Điền các ngày bị thiếu
df = fill_missing_dates(df, "PM25")  # Áp dụng cho một cột làm mẫu, các cột khác sẽ tự động bổ sung

# Hàm điền dữ liệu thiếu bằng KNN
def knn_impute(df, column_name):
    df['date_ordinal'] = df['Date'].map(pd.Timestamp.toordinal)
    knn_imputer = KNNImputer(n_neighbors=1)
    imputed_data = knn_imputer.fit_transform(df[['date_ordinal', column_name]])
    df[column_name] = imputed_data[:, 1]
    df.drop(columns=['date_ordinal'], inplace=True)
    return df

# Áp dụng KNN cho từng cột
columns_to_impute = ['PM25', 'PM10', 'CO', 'SO2', 'NO2', 'O3']
for col in columns_to_impute:
    df = knn_impute(df, col)

# Chuẩn hóa dữ liệu bằng MinMaxScaler
scaler = MinMaxScaler()
for col in columns_to_impute:
    df[f'{col}_scaled'] = scaler.fit_transform(df[[col]])

# Xuất dữ liệu sau xử lý
df.to_csv("D:/LeHuuKieu_processed.csv", index=False)